In [1]:
import json
import requests
from datetime import timedelta, datetime
from operator import itemgetter
import pandas as pd
from sklearn import linear_model, metrics
from sklearn import preprocessing
from scipy import stats
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

def createOlapPayload(sources,metrics,dimensions,startDate,endDate,filters):
	data = {}
	data['sources'] = sources
	data['metrics'] = metrics
	i = 1
	dimensionsDict = {}
	for dimension in dimensions:
		dimensionsDict[dimension] = i
		i += 1
	data['dimensions'] = dimensionsDict
	date1 = datetime.strptime(startDate,'%Y-%m-%d').strftime("%Y-%m-%d")
	date2 = (datetime.strptime(endDate,'%Y-%m-%d') + timedelta(1)).strftime("%Y-%m-%d")
	segments = []
	segmentsDict = {}
	segmentsDict['dateStart'] = "{0}T00:00:00+03:00".format(date1)
	segmentsDict['dateEnd'] = "{0}T00:00:00+03:00".format(date2)
	segmentsDict['filters'] = {}
	for filterKey, filterValue in filters.iteritems():
		segmentsDict['filters'][filterKey] = {}
		segmentsDict['filters'][filterKey]['type'] = 'DIMENSION'
		segmentsDict['filters'][filterKey]['ids'] = filterValue
	segments.append(segmentsDict)
	data['segments'] = segments
	return data

def getJSONOlapResponse(data):
	url = 'http://192.168.10.6:4000/api/OLAP/Plain'
	headers = {'Content-Type': 'application/json'}
	response = requests.post(url,data,headers=headers)
	return json.loads(response.text)

def transformOlapResponse(response,payload):
	data = []
	dimensionsList = [{'key': k, 'value': v} for k,v in payload['dimensions'].iteritems()]
	dimensionsList = sorted(dimensionsList, key=itemgetter('value'))
	for item in response:
		i = 0
		dataDict = {}
		for dimensionValue in item['dimensions']:
			dimensionName = dimensionsList[i]['key']
			dataDict[dimensionName] = dimensionValue
			i += 1
		metrics = item['value'][0]
		for k,v in metrics.iteritems():
			dataDict[k] = v
		data.append(dataDict)
	return data

In [2]:
filters = {'campaignTypeId': ['1','2'], 'campaignExternalId': ["16235643","16225787","16235646","15220880","15347849","13007173","1301025","14258995","10781855","13102809","13029234","13028622","15808246","15808840","17810872","16040960","16040964","16040962","15476048","16253530","14621066",
"15682873","11894512","10730032","14022527","13500701"]}
sources = ["direct_conversion","metrika"]
metrics = ["clicks","spent","shows","visits","cpaVisits"]
dimensions = ["dateDay"]
startDate = '2016-04-13'
endDate = '2016-03-27'
payload = createOlapPayload(sources,metrics,dimensions,startDate,endDate,filters)
response = getJSONOlapResponse(json.dumps(payload))
data = transformOlapResponse(response,payload)
df = pd.DataFrame(data)
df['dateDay'] = pd.to_datetime(df['dateDay'])

ValueError: No JSON object could be decoded

In [5]:
url = 'http://192.168.10.6:4000/api/OLAP/Plain'
headers = {'Content-Type': 'application/json'}
response = requests.post(url,json.dumps(payload),headers=headers)

In [10]:
json.dumps(payload)

'{"metrics": ["clicks", "spent", "shows", "visits", "cpaVisits"], "sources": ["direct_conversion", "metrika"], "segments": [{"dateStart": "2016-04-13T00:00:00+03:00", "dateEnd": "2016-03-28T00:00:00+03:00", "filters": {"campaignTypeId": {"type": "DIMENSION", "ids": ["1", "2"]}, "campaignExternalId": {"type": "DIMENSION", "ids": ["16235643", "16225787", "16235646", "15220880", "15347849", "13007173", "1301025", "14258995", "10781855", "13102809", "13029234", "13028622", "15808246", "15808840", "17810872", "16040960", "16040964", "16040962", "15476048", "16253530", "14621066", "15682873", "11894512", "10730032", "14022527", "13500701"]}}}], "dimensions": {"dateDay": 1}}'

In [11]:
response = requests.post(url,json.dumps(payload),headers=headers)

In [12]:
response.text

u'<!DOCTYPE html><html><head><title>Apache Tomcat/8.0.21 - Error report</title><style type="text/css">H1 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:22px;} H2 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:16px;} H3 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:14px;} BODY {font-family:Tahoma,Arial,sans-serif;color:black;background-color:white;} B {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;} P {font-family:Tahoma,Arial,sans-serif;background:white;color:black;font-size:12px;}A {color : black;}A.name {color : black;}.line {height: 1px; background-color: #525D76; border: none;}</style> </head><body><h1>HTTP Status 500 - org.glassfish.jersey.server.ContainerException: java.io.IOException: Server returned HTTP response code: 500 for URL: http://192.168.10.5:8083/druid/v2/</h1><div class="line"></div><p><b>type</b> Exception report</p><p><b>message

In [13]:
a = [1,2,3]
b = [4,5,6]
zip(a,b)

[(1, 4), (2, 5), (3, 6)]

In [15]:
[clicks*cpc for clicks,cpc in zip(a,b)]

[4, 10, 18]